In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/events.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Primero todo general- Como entraron a la página por primera vez? (con one hot enconding)

In [3]:
channel_general = df.groupby(['person','channel'])['timestamp'].min().to_frame().reset_index()
channel_general = channel_general.sort_values('timestamp', ascending=False).drop_duplicates(['person'])
channel_general = channel_general.pivot_table(index='person', columns='channel', values='timestamp', aggfunc='count').fillna(0)
channel_general = channel_general.add_prefix('fue_primero_')

# Ahora lo mismo para las últimas dos semanas 

In [4]:
df_acotado = df.loc[df['timestamp'] > pd.to_datetime('2018-05-15'), :]
user_features = df[['person']].drop_duplicates().copy().set_index('person')

In [5]:
channel_ult_15 = df_acotado.groupby(['person','channel'])['timestamp'].min().to_frame().reset_index()
channel_ult_15 = channel_ult_15.sort_values('timestamp', ascending=False).drop_duplicates(['person'])
channel_ult_15 = channel_ult_15.pivot_table(index='person', columns='channel', values='timestamp', aggfunc='count').fillna(0)
channel_ult_15 = channel_ult_15.add_prefix('fue_primero_en_ult_15_')

## csv

In [6]:
channel_general = user_features.join(channel_general).fillna(0)
channel_general.shape

(38829, 7)

In [7]:
# total
channel_general.to_csv("../data/feature_tipo_channel_primer_ingreso.csv", sep=',')

In [8]:
# ult 15 días
users = df[['person']].drop_duplicates().copy()
users.set_index('person', inplace=True)
user_features = users.join(channel_ult_15).fillna(0)
user_features.shape

(38829, 6)

In [9]:

user_features.to_csv("../data/feature_tipo_channel_primer_ingreso_ultimos_15_dias.csv", sep=',')